# MCP ToolSpec

This tool connects to MCP Servers and allows an Agent to call the tools provided by MCP Servers.

This idea is migrated from [Integrate MCP tools into LlamaIndex](https://psiace.me/posts/integrate-mcp-tools-into-llamaindex/).

To run this example, you need to edit the .env file to set the correct values for the [IPInfo API Token](https://ipinfo.io/signup) and a [OpenAI API key](https://platform.openai.com/api-keys).

```bash
# get the code
git clone https://github.com/run-llama/llama_index
cd llama_index/llama-index-integrations/tools/llama-index-tools-mcp/examples

# install dependencies
pip install ipinfo

cp .env.example .env
# NOTE: edit the .env file to have the correct values!

# run the server
python mcp_server.py --server_type=sse
```


In this example, we will create a toy example with an agent that can query the ip info of the user.

It's built using the `AgentWorkflow` class from LlamaIndex. If that's new to you, you can [read more about it](https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/)!

In [ ]:
from llama_index.llms.openai import OpenAI
import dotenv

dotenv.load_dotenv()

llm = OpenAI(model="gpt-4o")

In [ ]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent, ToolCallResult, ToolCall
from llama_index.core.workflow import Context

SYSTEM_PROMPT = """\
You are an AI assistant.

Before you help a user, you need to fetch the ip info first, to help you follow the laws of the country.
"""


async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can fetch the ip info of the user.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent


async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [ ]:
# Run the agent!
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  Hello! I need some help with my homework
Calling tool fetch_ipinfo with kwargs {'kwargs': ''}
Tool fetch_ipinfo returned meta=None content=[TextContent(type='text', text='{"ip": "71.17.163.153", "hostname": "71-17-163-153.regn.hsdb.sasknet.sk.ca", "city": "Regina", "region": "Saskatchewan", "country": "CA", "loc": "50.4501,-104.6178", "timezone": "America/Regina"}')] isError=False
Agent:  Hello! How can I assist you with your homework?


Here, we can see the agent is calling the `fetch_ipinfo` tool to get the ip info! This tool is running remotely on the mcp server.

The `MCPToolSpec` is connecting to the MCP server and creating `FunctionTool`s for each tool that is registered on the MCP server.

In [ ]:
tools = await mcp_tool.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

fetch_ipinfo Get the detailed information of a specified IP address

    Args:
        ip(str or None): The IP address to get information for. Follow the format like 192.168.1.1 .
        **kwargs: Additional keyword arguments to pass to the IPInfo handler.
    Returns:
        IPDetails: The detailed information of the specified IP address.
    


You can also limit the tools that the `MCPToolSpec` will create by passing a list of tool names to the `MCPToolSpec` constructor.

In [ ]:
mcp_tool = McpToolSpec(client=mcp_client, allowed_tools=["some fake tool"])
tools = await mcp_tool.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

---

**Alternatively**, 

You can directly use the `get_tools_from_mcp_url` or `aget_tools_from_mcp_url` functions to get a list of `FunctionTool`s from an MCP server.

In [ ]:
from llama_index.tools.mcp import (
    get_tools_from_mcp_url,
    aget_tools_from_mcp_url,
)

# async
tools = await aget_tools_from_mcp_url("http://127.0.0.1:8000/sse")

By default, this will use our `BasicMCPClient`, which will run a command or connect to the URL and return the tools.

You can also pass in a custom `ClientSession` to use a different client.

You can also specify a list of allowed tools to filter the tools that are returned.

In [ ]:
from llama_index.tools.mcp import BasicMCPClient

client = BasicMCPClient("http://127.0.0.1:8000/sse")

tools = await aget_tools_from_mcp_url(
    "http://127.0.0.1:8000/sse",
    client=client,
    allowed_tools=["fetch_ipinfo"],
)

Then create the agent directly using the obtained list of `FunctionTool`s.

In [ ]:
agent = FunctionAgent(
    name="Agent",
    description="An agent that can fetch the ip info of the user.",
    tools=tools,
    llm=llm,
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
# Run the agent!
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)